In [5]:
!pip3 install tira ir-datasets python-terrier
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt


In [6]:
# Assuming the text file contains one value per line
def read_text_file_to_array(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # Remove newline characters and convert to integers
            array = [(line.strip()) for line in lines]
            return array
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return None

# Read txt file and convert it to an array
file_path = "../20k.txt"
# 20k.txt contains the 20 000 most used english words
stopWordsArray = read_text_file_to_array(file_path)

# testing some stopwords
stopWordsArray = ["information", "retrieval"]
if stopWordsArray:
    print(f"Array from {file_path}: {stopWordsArray}")


Array from ../20k.txt: ['information', 'retrieval']


In [7]:
ensure_pyterrier_is_loaded()
dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

#TODO- currently we have to change path to 'index' on every run
indexer = pt.index.IterDictIndexer('/index10', stopwords= stopWordsArray)

indexref = indexer.index(dataset.get_corpus_iter(), meta={"docno", "text"})
index = pt.IndexFactory.of(indexref)

/tmp/ipykernel_513/1311260029.py:7: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  indexref = indexer.index(dataset.get_corpus_iter(), meta={"docno", "text"})
/usr/local/lib/python3.10/dist-packages/pyterrier/index.py:911: UserWarning: Indexing meta key text length requested 512 but exceeded in first document (actual length 1314). Increase the length in the meta dict for the indexer, e.g., pt.IterDictIndexer(..., meta={'text': 1314}).
  warn(msg)
ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:45<00:00, 2808.37it/s]


In [8]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25",metadata=["docno", "text"])

In [9]:
#dataset.get_topics('text').head(3)
#dataset.get_topics().head(1)
#dataset.get_qrels()

In [10]:
print('Now we do the retrieval...')
run = bm25(dataset.get_topics('text'))

print('Done. Here are the first 10 entries of the run')
run.head(10000)

Now we do the retrieval...


Done. Here are the first 10 entries of the run


,qid,docid,docno,text,rank,score,query
0,1,125137,1989.ipm_journal-ir0volumeA25A4.2,Improving the effectiveness of retrieval syste...,0,10.345954,retrieval system improving effectiveness
1,1,94858,2004.cikm_conference-2004.47,A multi-system analysis of document and term s...,1,10.317219,retrieval system improving effectiveness
2,1,102210,2012.wwwconf_conference-2012c.84,An early warning system for unrecognized drug ...,2,9.728031,retrieval system improving effectiveness
3,1,106738,2018.wwwconf_conference-2018c.102,How to Improve the Answering Effectiveness in ...,3,9.681757,retrieval system improving effectiveness
4,1,59634,D18-1292,Depth-bounding is effective: Improvements and ...,4,9.563852,retrieval system improving effectiveness
...,...,...,...,...,...,...,...
9995,10,100660,2020.chiir_conference-2020.8,Investigating Expectations for Voice-based and...,995,8.140603,how to represent natural conversations in word...
9996,10,111775,2020.trec_conference-2020.23,Glasgow Representation and Information Learnin...,996,8.140603,how to represent natural conversations in word...
9997,10,118165,2015.jasis_journal-ir0volumeA66A11.7,Learning from Elitist Jerks: Creating high-qua...,997,8.140603,how to represent natural conversations in word...
9998,10,43839,2021.gebnlp-1.11,Generating Gender Augmented Data for {NLP} Gen...,998,8.137689,how to represent natural conversations in word...


In [11]:
# run
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".


Done. run file is stored under "../runs/run.txt".
